### Написание красивых API с менеджерами контекста
Менеджеры контекста обладают достаточной гибкостью, и если к применению инструкции with подойти творчески, то для своих модулей и классов вы сможете определять удобные API.

Например, что, если «ресурсом», которым мы хотели бы управлять, являются уровни отступа текста в некоей программе — генераторе отчетов? Что, если бы для этого мы смогли написать исходный код, который выглядит вот так:

In [3]:
class Indenter:
    def __init__(self):
        self.level = 0
 
    def __enter__(self):
        self.level += 1
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1

    def print(self, text):
        print('  ' * self.level + text)

In [4]:
with Indenter() as indent:
    indent.print('привет!')
    with indent:
        indent.print('здорово')
        with indent:
            indent.print('бонжур')
    indent.print('эй')

  привет!
    здорово
      бонжур
  эй


### Поддержка инструкции with в собственных объектах
Вот пример простой реализации контекстного менеджера open():

In [5]:
class ManagedFile:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

Наш класс ___ManagedFile___ подчиняется протоколу менеджера контекста и теперь поддерживает инструкцию with.

In [6]:
with ManagedFile('hello.txt') as f:
    f.write('привет, мир!')
    f.write('а теперь, пока!')

Например, вы можете применить декоратор ___contextlib.contextmanager___, чтобы определить для ресурса фабричную функцию на основе генератора, которая затем будет автоматически поддерживать инструкцию ___with___. Вот как выглядит пример нашего контекстного менеджера ManagedFile, переписанный в соответствии с этим приемом:

In [7]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

In [8]:
with managed_file('hello.txt') as f:
    f.write('Привет, мир! ')
    f.write('А теперь, пока! ')

### Форматирование строковых значений

In [16]:
errno = 50159747054
name = 'Боб'

#### 1. «Классическое» форматирование строковых значений

In [20]:
'%s' % errno

'50159747054'

Используется спецификатор формата %x, чтобы преобразовать целочисленное значение в строковое и представить его как шестнадцатеричное число. Поскольку оператор % принимает всего один аргумент, вам необходимо обернуть правую часть в кортеж, как здесь:

In [21]:
'Эй, %s! Вот ошибка 0x%x!' % (name, errno)

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

Кроме того, к подстановкам переменных в своей форматной строке можно обращаться по имени. В этом случае в оператор % следует передать словарь с отображением имен на соответствующие им значения:

In [22]:
'Эй, %(name)s! Вот ошибка 0x%(errno)x!' % {"name": name, "errno": errno}

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

#### 2. «Современное» форматирование строковых значений
Функция format() может применяться для выполнения простого позиционного форматирования, точно так же, как вы могли поступать в случае с «классическим» форматированием:

In [23]:
'Привет, {}'.format(name)

'Привет, Боб'

Также вы можете обращаться к подстановкам переменных по имени и использовать их в любом порядке, в котором вы захотите. Это довольно мощное функциональное средство языка, поскольку оно позволяет изменять порядок следования отображаемых элементов, не изменяя аргументы, переданные в функцию форматирования:

In [27]:
'Эй, {n}! Вот ошибка 0x{err:x}!'.format(n=name, err=errno)

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

Этот пример также показывает, как изменился синтаксис форматирования целочисленной переменной в виде шестнадцатеричной строки. Теперь мы должны передавать спецификацию формата (format spec) путем добавления суффикса «:x» после имени переменной. Или вот использлвование суффикса «:b» для представления в бинарном виде:

In [30]:
'Эй, {n}! Вот ошибка bin:{err:b}!'.format(n=name, err=errno)

'Эй, Боб! Вот ошибка bin:101110101101110000001111111111101110!'

#### 3. Интерполяция литеральных строк (Python 3.6+)
Этот новый способ форматирования строк позволяет использовать выражения Python, которые встраиваются в строковые константы.

In [31]:
f'Привет, {name}!'

'Привет, Боб!'

В новом синтаксисе форматирования заложена большая мощь. Поскольку он позволяет встраивать произвольные выражения Python, вы даже можете выполнять локальные арифметические действия, как показано ниже:

In [32]:
a = 5
b = 10
f'Пять плюс десять равняется {a + b}, а не {2 * (a + b)}.'

'Пять плюс десять равняется 15, а не 30.'

За кадром форматированные строковые литералы представляют собой функциональное средство синтаксического анализатора Python, которое преобразовывает f-строки в серию строковых констант и выражений. Затем они объединяются, формируя окончательное строковое значение.

Предположим, что у нас есть следующая функция greet(), которая содержит f-строку:

In [33]:
def greet(name, question):
    return f"Привет, {name}! Как {question}?"

greet('Боб', 'дела')

'Привет, Боб! Как дела?'

Если эту функцию дизассемблировать и проинспектировать то, что происходит за кадром, то мы увидим ыот что:

In [34]:
import dis
dis.dis(greet)

  2           0 LOAD_CONST               1 ('Привет, ')
              2 LOAD_FAST                0 (name)
              4 FORMAT_VALUE             0
              6 LOAD_CONST               2 ('! Как ')
              8 LOAD_FAST                1 (question)
             10 FORMAT_VALUE             0
             12 LOAD_CONST               3 ('?')
             14 BUILD_STRING             5
             16 RETURN_VALUE


Строковые литералы также поддерживают существующий синтаксис форматных строк метода str.format(). Это позволяет решать те же самые задачи форматирования, которые мы обсудили в предыдущих двух разделах:

In [35]:
f"Эй, {name}! Вот ошибка {errno:#x}!"

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

In [36]:
f"Эй, {name}! Вот ошибка {errno:#b}!"

'Эй, Боб! Вот ошибка 0b101110101101110000001111111111101110!'

In [37]:
f"Эй, {name}! Вот ошибка {errno}!"

'Эй, Боб! Вот ошибка 50159747054!'

#### 4. Шаблонные строки
Еще один прием форматирования строк в Python представлен шаблонными строками. Этот механизм более простой и менее мощный, но в некоторых случаях он может оказаться именно тем, что вы ищете.

Давайте взглянем на простой пример приветствия:

In [39]:
from string import Template
t = Template('Эй, $name!')
t.substitute(name=name)

'Эй, Боб!'

Здесь вы видите, что нам приходится импортировать класс Template из встроенного модуля Python string. Шаблонные строки не являются ключевым функциональным свойством языка, но они обеспечиваются модулем стандартной библиотеки.

Еще одно отличие состоит в том, что шаблонные строки не допускают спецификаторы формата. Поэтому, чтобы заставить пример со строковой ошибкой работать, мы должны сами преобразовать целочисленный код ошибки в шестнадцатеричное строковое значение:

In [40]:
templ_string = 'Эй, $name! Вот ошибка $error!'
Template(templ_string).substitute(name=name, error=hex(errno))

'Эй, Боб! Вот ошибка 0xbadc0ffee!'

### Дзен Python от Тима Питерса
Красивое лучше, чем уродливое.  
Явное лучше, чем неявное.  
Простое лучше, чем сложное.  
Сложное лучше, чем запутанное.  
Плоское лучше, чем вложенное.  
Разреженное лучше, чем плотное.  
Читаемость имеет значение.  
Особые случаи не настолько особые, чтобы нарушать правила.  
При этом практичность важнее безупречности.  
Ошибки никогда не должны замалчиваться.  
Если не замалчиваются явно.  
Встретив двусмысленность, отбрось искушение угадать.  
Должен существовать один — и желательно только один — очевидный способ сделать это.  
Хотя он поначалу может быть и не очевиден, если вы не голландец.  
Сейчас лучше, чем никогда.  
Хотя никогда зачастую лучше, чем прямо сейчас.  
Если реализацию сложно объяснить — идея плоха.  
Если реализацию легко объяснить — идея, возможно, хороша.  
Пространства имен — отличная вещь! Давайте будем делать их больше!  

In [41]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


## 3. Эффективные функции
### 3.1. Функции Python — это объекты первого класса
Функции Python относятся к объектам первого класса. Их можно присваивать переменным, хранить их в структурах данных, передавать их в качестве аргументов другим функциям и даже возвращать их в качестве значений из других функций.

В будущем будем использовать простую функцию:

In [1]:
def yell(text):
    return text.upper() + '!'

yell('привет')

'ПРИВЕТ!'

In [2]:
bark = yell
bark('гав')

'ГАВ!'

In [3]:
bark.__name__

'yell'

#### Функции могут храниться в структурах данных
Поскольку функции — это объекты первого класса, их можно хранить в структурах данных точно так же, как это делается с другими объектами. Например, вы можете добавить функции в список:

In [4]:
funcs = [bark, str.lower, str.capitalize]
funcs

[<function __main__.yell(text)>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

Доступ к объектам-функциям, хранящимся внутри списка, осуществляется точно так же, как это происходит с объектом любого другого типа:

In [5]:
for f in funcs:
    print(f, f('всем привет'))

<function yell at 0x00000280341EEA60> ВСЕМ ПРИВЕТ!
<method 'lower' of 'str' objects> всем привет
<method 'capitalize' of 'str' objects> Всем привет


Хранящийся в списке объект-функцию даже можно вызвать без необходимости сначала присваивать его переменной. Для этого можно выполнить поиск и затем немедленно назвать результирующий «бестелесный» объект-функцию внутри одного-единственного выражения:

In [6]:
funcs[0]('приветище')

'ПРИВЕТИЩЕ!'

#### Функции могут передаваться другим функциям
Поскольку функции являются объектами, их можно передавать в качестве аргументов другим функциям. Вот функция ***greet***, которая форматирует строковое значение приветствия, используя переданный в нее объект-функцию, и затем его печатает:

In [7]:
def greet(func):
    greeting = func('Привет! Я — программа Python')
    print(greeting)

In [8]:
greet(bark)

ПРИВЕТ! Я — ПРОГРАММА PYTHON!


Разумеется, можно также определить новую функцию, чтобы сгенерировать приветствие с другим колоритом. Например, следующая функция whisper может сработать лучше:

In [9]:
def whisper(text):
    return text.lower() + '...'

greet(whisper)

привет! я — программа python...


Функции, которые в качестве аргументов могут принимать другие функции, также называются функциями более высокого порядка (higher-order functions). Они являются непременным условием функционального стиля программирования.

Классическим примером функций более высокого порядка в Python является встроенная функция map. Она принимает объект-функцию и итерируемый объект и затем вызывает эту функцию с каждым элементом итерируемого объекта, выдавая результат по мере прохождения итерируемого объекта.

Ниже показано, как вы могли бы отформатировать всю последовательность приветствий сразу, применив к ним функцию bark:

In [10]:
list(map(bark, ['здравствуй', 'эй', 'привет']))

['ЗДРАВСТВУЙ!', 'ЭЙ!', 'ПРИВЕТ!']

#### Функции могут быть вложенными
Быть может, вы удивитесь, но Python допускает определение функций внутри других функций. Такие функции нередко называются вложенными функциями (nested functions), или внутренними функциями (inner functions). Приведем пример:

In [11]:
def speak(text):
    def whisper_(t):
        return t.lower() + '...'
    return whisper(text)

speak('Привет, Мир')

'привет, мир...'

Итак, что же тут происходит? Всякий раз, когда вы вызываете функцию speak, она определяет новую внутреннюю функцию whisper_ и затем после этого немедленно ее вызывает. 

Правда, вот вам неожиданный поворот — функция whisper_ не существует за пределами функции speak:

In [12]:
whisper_('Йоу')

NameError: name 'whisper_' is not defined

In [13]:
speak.whisper_

AttributeError: 'function' object has no attribute 'whisper_'

Но что, если вы действительно хотите получить доступ к этой вложенной функции whisper_ за пределами функции speak? Не забывайте, функции являются объектами — и вы можете вернуть внутреннюю функцию источнику вызова родительской функции.

Например, ниже приведена функция, определяющая две внутренние функции. В зависимости от аргумента, передаваемого в функцию верхнего уровня, она выбирает и возвращает источнику вызова одну из внутренних функций:

In [14]:
def get_speak_func(volume):
    def whisper_(text):
        return text.lower() + '...'
    def yell_(text):
        return text.upper() + '!'
    if volume > 0.5:
        return yell_
    else:
        return whisper_

Обратите внимание на то, как функция get_speak_func фактически не вызывает ни одну из своих внутренних функций — она просто выбирает соответствующую внутреннюю функцию на основе аргумента volume и затем возвращает объект-функцию:

In [15]:
get_speak_func(0.3)

<function __main__.get_speak_func.<locals>.whisper_(text)>

In [16]:
get_speak_func(0.7)

<function __main__.get_speak_func.<locals>.yell_(text)>

Разумеется, вы можете продолжить и вызвать возвращенную функцию непосредственно, либо сначала присвоив ее переменной:

In [17]:
speak_func = get_speak_func(0.7)
speak_func('Привет')

'ПРИВЕТ!'

#### Функции могут захватывать локальное состояние
Чтобы это проиллюстрировать, можно немного переписать предыдущий пример функции get_speak_func. Новая версия сразу принимает аргументы «volume» и «text», чтобы немедленно сделать возвращаемую функцию вызываемой:

In [18]:
def get_speak_func(text, volume):
    def whisper_():
        return text.lower() + '...'
    def yell_():
        return text.upper() + '!'
    if volume > 0.5:
        return yell_
    else:
        return whisper_

In [19]:
get_speak_func('Привет, Мир', 0.7)()

'ПРИВЕТ, МИР!'

Теперь взгляните на внутренние функции whisper_ и yell_. Обратили внимание на то, что у них больше нет параметра text? Но каким-то непостижимым образом они по-прежнему могут получать доступ к этому параметру text, определенному в родительской функции. На самом деле они, похоже, захватывают и «запоминают» значение этого аргумента.

Функции, которые это делают, называются лексическими замыканиями (lexical closures) (или, для краткости, просто замыканиями). Замыкание помнит значения из своего лексического контекста, даже когда поток управления программы больше не находится в этом контексте.

В практическом плане это означает, что функции могут не только возвращать линии поведения, но и предварительно конфигурировать эти линии поведения. Ниже приведен еще один скелетный пример, который иллюстрирует эту идею:

In [21]:
def make_adder(n):
    def add(x):
        return x + n
    return add

plus_3 = make_adder(3)
plus_5 = make_adder(5)
print(plus_3(4))
print(plus_5(4))

7
9


#### Объекты могут вести себя как функции
Хотя в Python все функции являются объектами, обратное неверно. Объекты не являются функциями. Но они могут быть сделаны вызываемыми, что во многих случаях позволяет рассматривать их в качестве функций.

Если объект является вызываемым, то это означает, что вы можете использовать с ним синтаксис вызова функций с круглыми скобками и даже передавать в него аргументы вызова функции. Все это приводится в действие дандер-методом \_\_call\_\_ . Ниже приведен пример класса, определяющего вызываемый объект:

In [22]:
class Adder:
    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        return self.n + x

In [23]:
plus_3 = Adder(3)
plus_3(4)

7

За кадром «вызов» экземпляра объекта в качестве функции сводится к исполнению метода __call__ этого объекта.

Безусловно, не все объекты будут вызываемыми. Вот почему существует встроенная функция callable, которая проверяет, является объект вызываемым или нет:

In [26]:
print(callable(plus_3))
print(callable(yell))
print(callable('привет'))

True
True
False


### 3.2. Лямбды — это функции одного выражения

In [27]:
add = lambda x, y: x + y
add(5, 3)

8

Та же самая функция add может быть определена при помощи ключевого слова def, но она была бы чуть-чуть многословнее:

In [28]:
def add(x, y):
    return x + y

add(5, 3)

8

Функция для сортировки итерируемых объектов по альтернативному ключу:

In [29]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
sorted(tuples, key=lambda x: x[1])

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

In [30]:
sorted(tuples, key=lambda x: x[0])

[(1, 'd'), (2, 'b'), (3, 'c'), (4, 'a')]

In [31]:
sorted(tuples)

[(1, 'd'), (2, 'b'), (3, 'c'), (4, 'a')]

В приведенном выше примере мы сортируем список кортежей по второму значению в каждом кортеже. В этом случае лямбда-функция обеспечивает быстрый способ изменить порядок сортировки. Ниже приведен еще один пример сортировки, с которым вы можете поиграть:

In [32]:
sorted(range(-5, 6), key=lambda x: x * x)

[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]

Оба этих примера имеют в Python более сжатые реализации с использованием встроенных функций operator.itemgetter() и abs().

In [41]:
sorted(range(-5, 6), key=abs)

[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]

In [43]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
import operator
sorted(tuples, key=operator.itemgetter(1))

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

Вот еще одна интересная вещь о лямбдах: как и обычные вложенные функции, лямбды работают также и как ___лексические замыкания___.

Что такое лексическое замыкание? Это лишь затейливое название для функции, которая помнит значения из объемлющего лексического контекста, даже когда поток управления программы больше не находится в этом контексте. Вот (довольно академичный) пример, который иллюстрирует эту идею:

In [44]:
def make_adder(n):
    return lambda x: x + n

plus_3 = make_adder(3)
plus_5 = make_adder(5)
print(plus_3(4))
print(plus_5(4))

7
9


Например, делать что-то подобное для экономии пары строк кода просто глупо. Несомненно, технически все работает, и это вполне приличный «трюк». Но он порядочно смутит очередную девчонку или парня, которые в жесткие сроки должны отправить багфикс:

In [45]:
# Вредно:
class Car:
   rev = lambda self: print('Бум!')
   crash = lambda self: print('Бац!')

my_car = Car()
my_car.crash()

Бац!


In [46]:
my_car.rev()

Бум!


Похожие чувства и по поводу запутанных конструкций map() и filter() с использованием лямбд. Обычно гораздо лучше использовать конструкцию включения в список или в выражение-генератор:

In [47]:
# Вредно:
list(filter(lambda x: x % 2 == 0, range(16)))

[0, 2, 4, 6, 8, 10, 12, 14]

In [48]:
# Лучше:
[x for x in range(16) if x % 2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

In [50]:
# А ещё лучше так:
list(range(0,16,2))

[0, 2, 4, 6, 8, 10, 12, 14]

### 3.3. Сила декораторов

В своей основе декораторы Python позволяют расширять и изменять поведение вызываемых объектов (функций, методов и классов) без необратимой модификации самих вызываемых объектов.

In [54]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

In [55]:
@uppercase
def greet():
    return 'Привет!'

In [56]:
greet()

'ПРИВЕТ!'

In [57]:
# Нулевой декоратор - принимает и возвращает функцию
def null_decorator(func):
    return func

In [60]:
greet

<function __main__.uppercase.<locals>.wrapper()>

In [61]:
null_decorator(greet)

<function __main__.uppercase.<locals>.wrapper()>

In [62]:
uppercase(greet)

<function __main__.uppercase.<locals>.wrapper()>

In [64]:
def greet():
    return 'Привет!'

print(greet)
print(null_decorator(greet))
print(uppercase(greet))

<function greet at 0x00000280341EEC80>
<function greet at 0x00000280341EEC80>
<function uppercase.<locals>.wrapper at 0x00000280342D52F0>


#### Применение многочисленных декораторов к функции

In [65]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

Теперь давайте возьмем эти два декоратора и одновременно применим их к нашей функции greet. Для этого вы можете использовать обычный синтаксис @ и просто «уложить» многочисленные декораторы вертикально поверх одной-единственной функции:

In [66]:
@strong
@emphasis
def greet():
    return 'Привет!'

In [67]:
greet()

'<strong><em>Привет!</em></strong>'

Этот результат ясно показывает, в каком порядке декораторы были применены: снизу вверх. Сначала входная функция была обернута декоратором @emphasis, и затем результирующая (декорированная) функция снова была обернута декоратором @strong.

#### Декорирование функций, принимающих аргументы

Вот где на помощь приходят функциональные средства языка Python \*args и \*\*kwargs для работы с неизвестными количествами аргументов. Ниже приведен декоратор proxy, в котором задействуется их преимущество:

In [68]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

Давайте расширим прием, сформулированный декоратором proxy, в более полезный практический пример. Ниже приведен декоратор trace, который регистрирует аргументы функции и итоговые результаты, полученные во время исполнения:

In [72]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'ТРАССИРОВКА: вызвана {func.__name__}() '
              f'с {args}, {kwargs}')
        original_result = func(*args, **kwargs)
        print(f'ТРАССИРОВКА: {func.__name__}() '
              f'вернула {original_result!r}')
        return original_result
    return wrapper

In [70]:
@trace 
def say(name, line):
    return f'{name}: {line}'

In [73]:
say('Джейн', 'Привет, Мир')

ТРАССИРОВКА: вызвана say() с ('Джейн', 'Привет, Мир'), {}
ТРАССИРОВКА: say() вернула 'Джейн: Привет, Мир'


'Джейн: Привет, Мир'

#### Как писать «отлаживаемые» декораторы

При использовании декоратора вы на самом деле только подменяете одну функцию другой. Оборотной стороной этого процесса является то, что он «скрывает» некоторые метаданные, закрепленные за оригинальной (недекорированной) функцией.

Например, оригинальное имя функции, ее строка документации docstring и список параметров скрыты замыканием-оберткой:

In [74]:
def greet():
    """Вернуть дружеское приветствие."""
    return 'Привет!'

decorated_greet = uppercase(greet)

При попытке получить доступ к каким-либо из этих метаданных функции вместо них вы увидите метаданные замыкания-обертки:

In [75]:
greet.__name__

'greet'

In [76]:
greet.__doc__

'Вернуть дружеское приветствие.'

In [77]:
decorated_greet.__name__

'wrapper'

In [78]:
decorated_greet.__doc__

Это делает отладку и работу с интерпретатором Python неуклюжей и трудоемкой. К счастью, существует быстрое решение этой проблемы: декоратор functools.wraps, включенный в стандартную библиотеку Python.

Декоратор functools.wraps можно использовать в своих собственных декораторах для того, чтобы копировать потерянные метаданные из недекорированной функции в замыкание декоратора. Вот пример:

In [79]:
import functools
def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

Применение декоратора functools.wraps к замыканию-обертке, возвращаемому декоратором, переносит в него строку документации и другие метаданные входной функции:

In [81]:
@uppercase 
def greet():
    """Вернуть дружеское приветствие."""
    return 'Привет!'

In [82]:
greet.__name__

'greet'

In [83]:
greet.__doc__

'Вернуть дружеское приветствие.'

### 3.4. Веселье с \*args и \*\*kwargs
Они позволяют функции принимать необязательные аргументы, благодаря чему вы можете создавать гибкие API в модулях и классах:

In [84]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)

Приведенная выше функция требует по крайней мере одного аргумента под названием «required», то есть обязательный, но она также может принимать дополнительные позиционные и именованные аргументы.

Если мы вызовем функцию с дополнительными аргументами, то args соберет дополнительные позиционные аргументы в кортеж, потому что имя параметра имеет префикс \*.

Аналогичным образом, kwargs соберет дополнительные именованные аргументы в словарь, потому что имя параметра имеет префикс \*\*.

Когда мы вызываем функцию с различными комбинациями аргументов, вы видите, как Python собирает их в параметрах args и kwargs в соответствии с тем, являются они позиционными или именованными аргументами:

In [85]:
foo()

TypeError: foo() missing 1 required positional argument: 'required'

In [86]:
foo('привет')

привет


In [87]:
foo('привет', 1, 2, 3)

привет
(1, 2, 3)


In [88]:
foo('привет', 1, 2, 3, key1='значение', key2=999)

привет
(1, 2, 3)
{'key1': 'значение', 'key2': 999}


Название параметров args и kwargs принято по договоренности, как согласованное правило именования. Приведенный выше пример будет работать точно так же, если вы назовете их \*parms и \*\*argv. Фактическим синтаксисом является, соответственно, просто звездочка (\*) или двойная звездочка (\*\*).

#### Переадресация необязательных или именованных аргументов
Существует возможность передавать необязательные или именованные параметры из одной функции в другую. Это можно делать при помощи операторов распаковки аргументов \* и \*\* во время вызова функции, в которую вы хотите переадресовать аргументы.

Это также дает вам возможность модифицировать аргументы перед тем, как вы передадите их дальше. Вот пример:

In [90]:
def foo(x, *args, **kwargs):
    kwargs['имя'] = 'Алиса'
    new_args = args + ('дополнительный', )
    bar(x, *new_args, **kwargs)    

Данный прием может быть полезен для создания производных классов и написания оберточных функций. Например, он может применяться для расширения поведения родительского класса без необходимости повторять полную сигнатуру его конструктора в дочернем классе. Это может быть довольно удобно, если вы работаете с API, который может измениться за пределами вашего контроля:

In [91]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

class AlwaysBlueCar(Car):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.color = 'синий'

In [92]:
AlwaysBlueCar('зеленый', 48392).color

'синий'

In [93]:
Car('зеленый', 48392).color

'зеленый'

Еще один сценарий, где этот прием будет потенциально полезен, — написание оберточных функций, таких как декораторы. Там вы также захотите принимать произвольные аргументы, которые будут переправляться в обернутую функцию.

И если мы можем сделать это без необходимости копипастить сигнатуру оригинальной функции, то, возможно, сопровождение станет удобнее:

In [94]:
def trace(f):
    @functools.wraps(f)
    def decorated_function(*args, **kwargs):
        print(f, args, kwargs)
        result = f(*args, **kwargs)
        print(result)
    return decorated_function

@trace
def greet(greeting, name):
    return '{}, {}!'.format(greeting, name)

greet('Привет', 'Боб')

<function greet at 0x00000280342D5950> ('Привет', 'Боб') {}
Привет, Боб!


### 3.5. Распаковка аргументов функции

Действительно крутым, но немного загадочным функциональным средством языка является способность «распаковывать» аргументы функции из последовательностей и словарей при помощи операторов \* и \*\*.

Давайте определим простую функцию для работы в качестве примера:

In [95]:
def print_vector(x, y, z):
    print('<%s, %s, %s>' % (x, y, z))

Как вы видите, эта функция принимает три аргумента (x, y и z) и печатает их в приятно отформатированном виде. Мы можем применить эту функцию в нашей программе для структурной распечатки трехмерных векторов:

In [96]:
print_vector(0, 1, 0)

<0, 1, 0>


Нужно сказать, что в зависимости от того, какую структуру данных мы выбираем для представления трехмерных векторов, их распечатка нашей функцией print_vector может восприниматься немного неуклюжей. Например, если наши векторы представлены в виде кортежей или списков, то во время их распечатки мы должны явным образом задавать индекс каждого компонента:

In [97]:
tuple_vec = (1, 0, 1)
list_vec = [1, 0, 1]
print_vector(tuple_vec[0], tuple_vec[1], tuple_vec[2])

<1, 0, 1>


К счастью, в Python имеется более подходящий способ справиться с этой ситуацией — при помощи распаковки аргументов функции с использованием оператора \*:

In [98]:
print_vector(*tuple_vec)
print_vector(*list_vec)

<1, 0, 1>
<1, 0, 1>


Размещение звездочки \* перед итерируемым объектом в вызове функции его распакует и передаст его элементы как отдельные позиционные аргументы в вызванную функцию.

Этот прием работает для любого итерируемого объекта, включая выражения-генераторы. В результате использования оператора \* с генератором все поступающие из генератора элементы будут использованы и переданы в функцию:

In [99]:
genexpr = (x * x for x in range(3))
print_vector(*genexpr)

<0, 1, 4>


Помимо оператора \* для распаковки последовательностей, в частности кортежей, списков и генераторов, в позиционные аргументы, также имеется оператор \*\* для распаковки именованных аргументов, поступающих из словарей. Предположим, что наш вектор был представлен в виде следующего объекта dict:

In [100]:
dict_vec = {'y': 0, 'z': 1, 'x': 1}

Этот объект-словарь можно передать в функцию print_vector практически таким же образом, использовав оператор \*\* для распаковки:

In [101]:
print_vector(**dict_vec)

<1, 0, 1>


Поскольку словари не упорядочены, этот оператор соотносит значения словаря и аргументы функции на основе ключей словаря: аргумент x получает значение, связанное в словаре с 'x'.

Если для распаковки словаря использовать оператор одинарной звездочки (\*), то вместо этого ключи будут переданы в функцию в произвольном порядке:

In [104]:
print_vector(*dict_vec)

<y, z, x>


In [105]:
dict_vec = {'z': 0, 'y': 1, 'x': 1}
print_vector(*dict_vec)

<z, y, x>


### 3.6. Здесь нечего возвращать
В конец любой функции Python добавляет неявную инструкцию return None. По этой причине, если в функции не указано возвращаемое значение, по умолчанию она возвращает None.

Это означает, что инструкции return None можно заменять на пустые инструкции return или даже пропускать их полностью и по-прежнему получать тот же самый результат:

In [106]:
def foo1(value):
    if value:
        return value
    else:
        return None

def foo2(value):
    """Пустая инструкция return подразумевает `return None`"""
    if value:
        return value
    else:
        return

def foo3(value):
    """Пропущенная инструкция return подразумевает `return None`"""
    if value:
        return value

Все три функции правильно возвращают None, если передать им в качестве единственного аргумента фиктивное значение:

In [107]:
type(foo1(0))

NoneType

In [108]:
type(foo2(0))

NoneType

In [109]:
type(foo3(0))

NoneType